$ \hat{H}_{tr} = \omega_r \hat{a}^\dagger \hat{a} + \hat{H}_t - i g (\hat{n}_t - n_g)(\hat{a} - \hat{a}^\dagger) $

In [ ]:

import scqubits as scq
import qutip as qt
import numpy as np


EC = 0.2
EJ = 15 # 0.22 * 110
E_osc = 6.8  # GHz, resonator frequency

tmon = scq.Transmon(
    EJ=EJ,      # Josephson energy
    EC=EC,      # Charging energy
    ng=0.3,     # Offset charge
    ncut=41,    # Charge basis cutoff
    truncated_dim=10  # Number of levels to keep
)


resonator = scq.Oscillator(E_osc=E_osc, truncated_dim=20)

hilbertspace = scq.HilbertSpace([tmon, resonator])

g = -0.120*1j #g = 120 MHz, calculate this from  capacitance matrix

hilbertspace.add_interaction(
    expr="g * n * a - g * n * adag",
    op1=("n", tmon.n_operator),
    op2=("adag", resonator.creation_operator),
    op3 = ("a", resonator.annihilation_operator),
    add_hc=False
)

hilbertspace.generate_lookup()
    
    # Extract eigenvalues
evals = hilbertspace["evals"][0]
diag_dressed_hamiltonian = (
    2 * np.pi * qt.Qobj(np.diag(evals), dims=[hilbertspace.subsystem_dims] * 2)
    )

total_truncation = 10
def truncate(operator: qt.Qobj, dimension: int) -> qt.Qobj:
        return qt.Qobj(operator[:dimension, :dimension])
    
diag_dressed_hamiltonian_trunc = truncate(diag_dressed_hamiltonian, total_truncation)
evalues = (diag_dressed_hamiltonian_trunc.eigenenergies() - diag_dressed_hamiltonian_trunc.eigenenergies()[0]) / 6.28
    
    # Compute chi
e_11 = evalues[hilbertspace.dressed_index((1,1))]
e_10 = evalues[hilbertspace.dressed_index((1,0))]
e_01 = evalues[hilbertspace.dressed_index((0,1))]
e_00 = evalues[hilbertspace.dressed_index((0,0))]
    
chi = e_11 - e_10 - e_01 + e_00



print(f"{chi*1000:.4f} Chi (MHz)")

print(f"{e_11:.4f}, {e_10:.4f}, {e_01:.4f}, {e_00:.4f} Energies (GHz) for (11, 10, 01, 00) states")
detuning = e_10 - e_01
# print(f"{detuning:.4f} Detuning (GHz)")


Chi = abs(chi)

-2.6408 Chi (MHz)
11.4894, 4.6805, 6.8116, 0.0000 Energies (GHz) for (11, 10, 01, 00) states
